#### Feature Extraction

In [7]:
#%store <var_name>
#%store -r <var_name>

In [13]:
from keras import  models
# load VGG-16 model: 23 layers, 138,357,544 params, 528MB
from keras.applications.vgg16 import VGG16
# load and evaluate a saved model 
#import tensorflow as tf
#from tensorflow import keras
from keras.models import load_model

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Imports from Sci-Kit Learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# Load data from previous notebooks
%store -r dog_data_part2
%store -r X_valid
%store -r y_valid

In [19]:
len(X_valid)
len(y_valid)

43733

In [21]:
set(y_valid)

{0, 1}

In [16]:
X_valid[0].shape

(224, 224, 3)

In [7]:
# Load target labels for validation data set
try:
    with open('../../assets/y_validation_labels.txt', 'r') as validlabels:
        # Store validation images as variable for loading into model for scoring
        y_valid_loaded = np.array([valid_image_label for valid_image_label in validlabels])

    # Load feature matrix of Validation images from text file
    with open('../../assets/X_validation_image_data.txt', 'r') as validimages:
        # Store validation images as variable for loading into model for scoring
        X_valid_loaded = np.array([processed_image for processed_image in validimages])

    # Check that all is copacetic
    type(dog_data_part2), dog_data_part2[0], type(dog_data_part2[0])
    # Length of list of images
    print('Final Length X_valid: ', len(X_valid_loaded)) 
    
# Just in case the data can be loaded from text, we want to know!
except FileNotFoundError:
    print('HEADS UP: Could not load validation images/labels from .txt files...\n\
                \t...return to previous notebooks, store pre-processed data, and re-run this cell.')
    print()

Final Length X_valid:  2142917


In [11]:
type(X_valid_loaded[0])

numpy.str_

In [9]:
# https://keras.io/api/applications/vgg/#vgg16-function
# https://towardsdatascience.com/a-demonstration-of-transfer-learning-of-vgg-convolutional-neural-network-pre-trained-model-with-c9f5b8b1ab0a

# Adjust input size of the model for include_top=False
# new_input = Input(shape=(224,224,3)) --> MUST INCLUDE THIS PARAM AFTER CLASSIFICATION

# load the model weights into memory
base_model = VGG16(
                include_top=True,   # include_top=False to load model wihtout the fully-connected output layers used to make predictions
                weights="imagenet", # Weights are downloaded automatically when instantiating a model: Keras Applications ~/.keras/models/
                input_tensor=None,  # input_tensor = new_input = Input(shape=(224,224,3))
                input_shape=None,
                pooling=None,
                classes=1000,
                classifier_activation="softmax",
            )

# Cut-Off the VGG-16 Model after the last Conv2D layer (18)
dogg16 = models.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('flatten').output
                          )

# Extract features
#flatten_features = model_vgg16.predict(x)
# save model and architecture to single file
dogg16.save('../../model_dogg16.h5') 
print("Saved model to disk")

model.add(Dense(512, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile('adam', #optimizer=
              loss="categorical_crossentropy", # 'binary_crossentropy'
              metrics=["accuracy"])

# Summarize the loaded model after dropping the two fully connected layers and the output layer
dogg16.summary()

Saved model to disk
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None

#### Notes from General Assembly-DSI Lessons 
4.01-lesson-logistic-regression/
<br>4.02-lesson-regularization/
<br>4.03-lesson-knn/
<br>4.04-lesson-classification_metrics_i/
<br>4.05-lesson-classification-metrics-ii/
<br>4.06-lesson-hyperparameters-gridsearch-and-pipelines/
<br>4.07-lesson-hypothesis-testing/

In [ ]:
history = model.fit(x=train_imgs_scaled, 
                    y=train_labels_enc,
                    validation_data=(validation_imgs_scaled, validation_labels_enc),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

In [11]:
# EDA might be handy to use:
# 1. .where()
# 2. .value_counts(normalize = True)
# 3. .astype(int)
#EXAMPLE: Turn outcome to 1 if success and 0 if failure
df['outcome'] = np.where(df['target'] == 'Success Condition', 1, 0)

In [ ]:
# Step 0: Assemble X and y
features = ['feat1', 'feat2'] # predictor variables {x}
X = df[features]; # predictor variables
y = df[ 'target' ]; # our target

In [ ]:
# load weights into new model
loaded_model = '../../model_vgg16_flatten.h5'

# Load base model weights
loaded_model = load_model(loaded_model)
#loaded_model.load_weights("~\.keras\models.h5") 
print("Loaded model from disk")

In [ ]:
loaded_model[0]

In [ ]:
weights_path